# **Predicting Fire Radiative Power Using Random Forest Regressor**



## **Introduction:**

This notebook explores the prediction Fire Radiative Power (FRP) using satellite data of Australian bushfires with a Random Forest Regressor. The goal of this notebook is to predict the intensity of wildfires based on features like brightness values (`bright_ti4` and `bright_ti5`), geographical coordinates, and confidence levels. The analysis covers data preprocessing, feature selection, model training, and evaluation.

## **Importing Required Libraries**

The following libraries are essential for data processing, model training, and visualization:

- **`os`**: Handles file paths and directories.
- **`pandas`**: Used for data manipulation and analysis.
- **`numpy`**: Core library for numerical computing.
- **`matplotlib.pyplot`**: Creates static visualizations, like plots and charts.
- **`seaborn`**: Built on `matplotlib`, provides high-level statistical graphics.
- **`sklearn.ensemble.RandomForestRegressor`**: Random Forest model for handling non-linear relationships.
- **`sklearn.model_selection.train_test_split`**: Splits the dataset into training and testing subsets.
- **`sklearn.metrics`**: Provides performance metrics like MSE and R² score.
- **`geopandas`**: Handles geographic data for plotting maps of Australia.
- **`matplotlib.colors`**: Manages color normalization for visualizations.

In [3]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import geopandas as gpd
import matplotlib.colors as mcolors